# Calculating quadratic density $\eta(p_k)$ from $\Delta H(p)$
The range of the primes in $\mathcal{G}(23^\#)$ is almost covered by the horizon of survival $H(p_k)= [p_{k+1},p_{k+1}^2]$
for $p_k=14929$
$$ H(p=14929) \; {\rm with} \; \lambda=0.3880218443549397$$

Continuing our studies of $\Delta H(p_k)$, here we study the quadratic density $\eta(p)$ of the intervals of survival.
$$ \eta(p_k) = \frac{1}{p_{k+1}-p_k} |N_{\Delta H}(p_k)|$$
This gives us the average occurrence of a gap $g$ in an interval $[n^2,(n+1)^2]$ in $\Delta H(p_k)$.  

In [1]:
%reset -f

import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import polyval
import array
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
#from plotnine import ggplot, geom_point, aes, geom_line, theme, ggsave
import gc
import psutil
import sys

import itertools
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
plt.ion


<function matplotlib.pyplot.ion() -> 'AbstractContextManager'>

In [3]:
try:
    primes23 = np.load('primes23.npy')
except FileNotFoundError:
    print("File primes23.npy not found.  See 01 notebooks.")
    sys.exit(1)

try:
    G23 = np.load('G23uint.npy')
except FileNotFoundError:
    print("File G23uint.npy not found.  See 01 notebooks.")
    sys.exit(2)

try:
    cG23 = np.load('cG23.npy')
except FileNotFoundError:
    print("File cG23.npy not found.  See 01 notebooks.")
    sys.exit(3)


In [4]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

Available memory: 3111.70 MB


In [5]:
lenp23 = len(primes23)
maxp23 = primes23[-1]
print(f"Length primes {lenp23} gaps {len(cG23)} maxp {maxp23}")
# These lengths= primes 12283522 gaps 12283523 maxp 223092827

Length primes 12283522 gaps 12283523 maxp 223092827


In [6]:
lambda23 = np.zeros(lenp23)

In [7]:
# primes23[2:5] are 37, 41, 43
lambda23[0] = 1
ilam = 0
ip = 2 # the offset from lambda23 into primes23 for p0=37
while (ip < (lenp23-1)):
    ilam += 1
    ip += 1
    lambda23[ilam] = lambda23[ilam-1]*(primes23[ip]-3)/(primes23[ip]-2)

In [8]:
# to match indexing across arrays, cG23[i]=primes23[i]-primes23[i-1]
cG23[0:10]

array([28,  2,  6,  4,  2,  4,  6,  6,  2,  6], dtype=uint16)

In [9]:
lambda23[-100:-1]

array([0.19416066, 0.19416066, 0.19416065, 0.19416065, 0.19416065,
       0.19416065, 0.19416065, 0.19416065, 0.19416065, 0.19416065,
       0.19416065, 0.19416065, 0.19416065, 0.19416064, 0.19416064,
       0.19416064, 0.19416064, 0.19416064, 0.19416064, 0.19416064,
       0.19416064, 0.19416064, 0.19416064, 0.19416064, 0.19416064,
       0.19416063, 0.19416063, 0.19416063, 0.19416063, 0.19416063,
       0.19416063, 0.19416063, 0.19416063, 0.19416063, 0.19416063,
       0.19416063, 0.19416062, 0.19416062, 0.19416062, 0.19416062,
       0.19416062, 0.19416062, 0.19416062, 0.19416062, 0.19416062,
       0.19416062, 0.19416062, 0.19416062, 0.19416061, 0.19416061,
       0.19416061, 0.19416061, 0.19416061, 0.19416061, 0.19416061,
       0.19416061, 0.19416061, 0.19416061, 0.19416061, 0.1941606 ,
       0.1941606 , 0.1941606 , 0.1941606 , 0.1941606 , 0.1941606 ,
       0.1941606 , 0.1941606 , 0.1941606 , 0.1941606 , 0.1941606 ,
       0.1941606 , 0.19416059, 0.19416059, 0.19416059, 0.19416

In [10]:
maxgap = max(cG23)
maxgap

np.uint16(248)

In [11]:
np.mean(cG23)

np.float64(18.161961352618462)

In [12]:
np.sqrt(maxp23)

np.float64(14936.292277536617)

In [13]:
# bounds on parameters for cG23
i=0
while (primes23[i] < 14936):
    i += 1
i -= 1
print(f"i {i} p23 {primes23[i]} p^2 {primes23[i]**2} maxp23 {primes23[-1]} lambda {lambda23[i-2]:.4f}")

i 1738 p23 14929 p^2 222875041 maxp23 223092827 lambda 0.3880


In [14]:
# bounds on parameters for cG29
pbound = np.sqrt(29)*14936
i=0
while (primes23[i] < pbound):
    i += 1
i -= 1
print(f"i {i} p23 {primes23[i]} p^2 {primes23[i]**2} maxp23 {primes23[-1]} lambda {lambda23[i-2]:.4f}")

i 7863 p23 80429 p^2 6468824041 maxp23 223092827 lambda 0.3304


## Accumulations over intervals $\Delta H(p)$
For comparison with the relative populations $w_{g,1}(p^\#)$, we accumulate the counts of gaps within the intervals of survival $\Delta H(p) = [p^2, q^2]$.

In [15]:
ninterval = 1738 # max prime index for cG23
ngaps = int(maxgap/2)
DelHcounts = np.zeros((ninterval, ngaps), dtype=int)
print(f"array created {ninterval} x {ngaps}")

array created 1738 x 124


In [16]:
# NOTE on indexing:  cG23[i] = primes23[i]-primes23[i-1]
ip = 0
p = primes23[ip]
psqr = p*p
i = 0
while (primes23[i] < psqr):
    i +=1
# We're ready to start the count - i is pointing at the first gap in the first interval
while (ip < ninterval):
    print(f"ip {ip} of {ninterval} p= {primes23[ip]} at i {i}", end='\r')
    psqr = (primes23[ip+1])**2
    while (primes23[i] < psqr):
        j = int(cG23[i]/2 - 1+0.1)  # indices are half the value of the gap, adding 0.1 to insure rounding
        DelHcounts[ip,j] = DelHcounts[ip,j] + 1
        i += 1
    ip += 1
# Counts of DelH completed over the covered range, from p=29 to p=14929


ip 1737 of 1738 p= 14923 at i 12262909

In [17]:
# np.save('DelH29_14923.npy',DelHcounts)
# visual checks on histogram data
DelHcounts[-11:,0:10]

array([[ 412,  432,  753,  327,  400,  504,  314,  243,  420,  233],
       [1278, 1272, 2217, 1011, 1273, 1676,  877,  670, 1302,  680],
       [ 879,  845, 1520,  674,  885, 1082,  605,  425,  862,  408],
       [1647, 1753, 2976, 1327, 1735, 2221, 1255,  906, 1649,  889],
       [ 216,  225,  357,  158,  209,  280,  158,  113,  222,  112],
       [1090, 1055, 1899,  796, 1073, 1358,  757,  582, 1086,  556],
       [ 854,  821, 1577,  661,  840, 1084,  553,  480,  805,  451],
       [ 438,  428,  757,  326,  425,  577,  292,  253,  368,  225],
       [ 636,  644, 1104,  521,  665,  786,  481,  350,  638,  333],
       [2768, 2804, 4911, 2174, 2839, 3509, 2022, 1551, 2707, 1471],
       [ 634,  636, 1148,  496,  622,  830,  405,  323,  580,  338]])

In [18]:
# visual checks on histogram data
sum(DelHcounts[:,])

array([ 895223,  895902, 1575342,  688899,  888391, 1123493,  615227,
        455120,  818700,  435767,  377389,  558733,  260987,  283355,
        494297,  152949,  160426,  257558,  118599,  139438,  200812,
         81339,   69001,  118137,   66068,   49069,   81284,   41034,
         35534,   70642,   21417,   22202,   39803,   16013,   22638,
         22275,   10993,    9399,   18051,    8834,    6265,   12858,
          4509,    4594,    9254,    2981,    2698,    4697,    2438,
          2542,    3210,    1573,    1235,    2185,    1276,     996,
          1481,     608,     578,    1299,     390,     416,     718,
           251,     297,     406,     165,     136,     294,     188,
           115,     172,      67,      86,     150,      67,      57,
            81,      33,      38,      42,      26,      15,      43,
            22,      12,      14,      17,      14,      13,      10,
             9,       5,       1,       1,       5,       2,       3,
             6,     

## Quadratic density $\eta(p_k)$
From the counts of gaps in the intervals of survival $\Delta H(p_k)$ we calculate the quadratic residues
$$\eta(p_k) = \frac{1}{p_{k+1}-p_k} \cdot N_{\Delta H}(p_k).$$
As $p_k$ grows the length of the interval $\Delta H(p_k)$ grows almost proportionally to the gap $g=p_{k+1}-p_k $.
$$|\Delta H(p_k)| = p_{k+1}^2- p_k^2 = g \cdot(2p_k+g)$$


In [19]:
# calculate the quadratic density eta across the intervals of survival
eta = np.zeros((ninterval,ngaps),dtype=float)
i=0
while (i < ninterval):
    j=0
    while (j < ngaps):
        eta[i,j] = DelHcounts[i,j]/(cG23[i+1])
        j += 1
    i += 1

In [20]:
# checking details in small data samples
i=1600
while (i < 1620):
    print(f"prime {primes23[i]} gap {cG23[i+1]} count {DelHcounts[i,0:4]} densities {[f'{num:.3f}' for num in eta[i,0:4]]}")
    i += 1

prime 13619 gap 8 count [ 760  783 1380  619] densities ['95.000', '97.875', '172.500', '77.375']
prime 13627 gap 6 count [ 582  629 1068  461] densities ['97.000', '104.833', '178.000', '76.833']
prime 13633 gap 16 count [1594 1629 2887 1241] densities ['99.625', '101.812', '180.438', '77.562']
prime 13649 gap 20 count [1997 1989 3476 1545] densities ['99.850', '99.450', '173.800', '77.250']
prime 13669 gap 10 count [1041  937 1763  715] densities ['104.100', '93.700', '176.300', '71.500']
prime 13679 gap 2 count [199 198 348 160] densities ['99.500', '99.000', '174.000', '80.000']
prime 13681 gap 6 count [ 614  599 1011  489] densities ['102.333', '99.833', '168.500', '81.500']
prime 13687 gap 4 count [431 412 697 321] densities ['107.750', '103.000', '174.250', '80.250']
prime 13691 gap 2 count [193 175 382 168] densities ['96.500', '87.500', '191.000', '84.000']
prime 13693 gap 4 count [388 360 656 318] densities ['97.000', '90.000', '164.000', '79.500']
prime 13697 gap 12 count [1

In [21]:
# Develop a master color dictionary - up to g=100
# we set colors by family, as determined by prime factors of the gap
mastercolordict={'2':'#FF0000', '4':'#FF8888', '8':'#EEBBBB', '16':'#FF99CC', '32':'#FFDDDD', '64':'#FFEEEE',
                 '6':'#0000FF', '12':'#00DDFF', '18':'#6666FF', '24':'#BBBBFF', '36':'#DDDDFF', '48':'#0000AF',
                 '54':'#44448F', '72':'#88888F',
                 '10':'#00CC00', '20':'#98FB98', '40':'#DDFFCF', '50':'#BBFFBB', '80':'#44AA66', '100':'#228F3F',
                 '30':'#FFD700', '60':'#FFB000', '90':'#FFCC33',
                 '14':'#DD00EE', '28':'#AA22CC', '56':'#660066', '98':'#662266',
                 '42':'#0088BB', '84':'#4488BB', '70':'#00BB88', 
                 '22':'#884400', '44':'#884444', '66':'#884488', '88':'#AA6644',
                 '26':'#AAAAAA', '52':'#888888', '78':'#6666AA',
                 '34':'#CC0000', '68':'#CC4444', '38':'#AA8800', '76':'#AA9944'}

In [22]:
gapnames = np.arange(2, maxgap+2, 2)

In [23]:
print(f"max {ninterval} {primes23[ninterval]}")

max 1738 14929


In [24]:
lowgapdex = 0
highgapdex = 45  # the gap index is half the gap size:  gap = 2*(gapdex+1)
gaprange = (gapnames[lowgapdex:highgapdex]).astype(str)
numgaps = highgapdex-lowgapdex

def draw_eta(numpdex, highpdex):   # the input parameters are indices, not the prime values themselves
    if (numpdex < highpdex):
        lowpdex = highpdex - numpdex
    else:
        lowpdex = 0
        highpdex = numpdex

    primerange = (primes23[lowpdex:highpdex]).astype(str)
    partialeta = eta[lowpdex:highpdex, lowgapdex:highgapdex].copy()
    partialeta = partialeta.transpose()
    
    flat_eta = partialeta.flatten()
    xgaps = np.repeat(gaprange, numpdex)

    # create the color dictionary, using gapsizes in primes23 from lowpdex to highpdex
    i = 0
    colordict = []
    while (i < highgapdex):
        j=lowpdex
        while (j < highpdex):
            pstring = str(primes23[j])
            gapstring = str(cG23[j+1])  # cG23[i] = primes23[i]-primes[i-1]
            if gapstring in mastercolordict:
                colorstring = mastercolordict[gapstring]
            else:
                print(f"Missing color for {gapstring} at {pstring}")
                colorstring = '#080808'
            colordict.append(colorstring)
            j += 1
        i += 1

    fig, ax = plt.subplots()
    fig.set_size_inches(12,8)

    lamvalue0 = lambda23[lowpdex]
    lamvalue1 = lambda23[highpdex]
    middex = int((highpdex+lowpdex)/2)
    ptitle = (primes23[middex])**2
    ax.set_title(f"Quadratic densities over prime gaps in {numpdex} intervals of survival around {ptitle:.3e}\n primes {primes23[lowpdex]}-{primes23[highpdex]},  $\lambda \in$ [{lamvalue1:.3f},{lamvalue0:.3f}]")
    plt.grid(True,axis='y',color='#A0A0A0',lw=0.4)

    eta_mu = np.zeros(numgaps)
    eta_std = np.zeros(numgaps)
    i=0
    while (i < numgaps):
        eta_mu[i] = np.mean(partialeta[i,:])
        eta_std[i] = np.std(partialeta[i,:])
        i += 1

    eta_stdmark0 = eta_mu - eta_std
    eta_stdmark00 = eta_mu - 2*eta_std
    eta_stdmark1 = eta_mu + eta_std
    eta_stdmark11 = eta_mu + 2*eta_std

    plt.scatter(gaprange,eta_mu, marker='D', color='black', s=90)
    plt.scatter(gaprange,eta_stdmark0, marker='_', color='black', s=150)
    plt.scatter(gaprange,eta_stdmark1, marker='_', color='black', s=150)
    plt.scatter(gaprange,eta_stdmark00, marker='_', color='black', s=150)
    plt.scatter(gaprange,eta_stdmark11, marker='_', color='black', s=150)
    plt.scatter(xgaps,flat_eta, c=colordict, marker='x', s=25)

    plt.show()

# Interactive controls

xnumpSelect = widgets.IntSlider(min=10, max=200, step=5, value=25,
                  description="Num DelH", layout=widgets.Layout(width='80%'), disabled=False)
xhighpSelect = widgets.IntSlider(min=10, max=1737, step=1, value=1737, description="High_pdex", layout=widgets.Layout(width='80%'), disabled=False)

interact(draw_eta, numpdex=xnumpSelect, highpdex=xhighpSelect)


interactive(children=(IntSlider(value=25, description='Num DelH', layout=Layout(width='80%'), max=200, min=10,…

<function __main__.draw_eta(numpdex, highpdex)>

## Quadratic density for intervals of survival $\Delta H(p)$
The interactive figure above displays the quadratic density of prime gaps by size of gap, across consecutive intervals of survival $\Delta H(p_k) = [p_k^2, p_{k+1}^2]$.

We color-code the marker for $\Delta H(p_k)$ by the size of the gap $p_{k+1}-p_k$.  For each gap we also show the mean of the samples and two standards deviations above and below the mean.
